# 5VALIDACION: VALIDACIÓN Y SELECCIÓN FINAL

Este notebook demuestra el funcionamiento del módulo 5validacion, que valida las carteras optimizadas y calcula métricas finales para seleccionar la mejor estrategia.

In [ ]:
import sys
import importlib
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importar módulos (los nombres que empiezan con números requieren importlib)
datos = importlib.import_module('1datos')
markowitz = importlib.import_module('2markowitz')
factores = importlib.import_module('3factores')
multifactorial = importlib.import_module('4multifactorial')
validacion = importlib.import_module('5validacion')

print("Librerías importadas correctamente")

In [ ]:
# Cargar datos y preparar todas las estrategias
ruta_csv = '../data/prod_long_sharpe_u50_20260116_v5_train_dataset.csv'
retornos = datos.cargar_retornos(ruta_csv)
preparador = datos.PreparadorDatos(retornos, rf_anual=0.02)
preparador.calcular_estadisticas()
mu, Sigma, rf = preparador.obtener_estadisticas()

# Construir señales
stats = datos.calcular_estadisticas_basicas(retornos)
momentum = factores.calcular_momentum_12_2(retornos)
mom_actual = momentum.iloc[-1].dropna()
volatilidades = factores.calcular_volatilidad_rolling(retornos)
vol_actual_63d = volatilidades['vol_63d'].iloc[-1].dropna()
betas = factores.calcular_betas(retornos)

senales = {
    'momentum': mom_actual,
    'vol_63d': -vol_actual_63d,
    'beta': -betas,
    'sharpe_hist': stats['sharpe_historico']
}
senales_norm = factores.normalizar_senales(senales)
X, nombres_factores = factores.construir_matriz_caracteristicas(senales_norm)
W_k = multifactorial.calcular_pesos_exposicion(X, metodo='inverso_varianza')

# Crear estrategias
cartera_max_sharpe = markowitz.optimizar_sharpe_maximo(mu, Sigma, rf)

b_star = np.array([0.5, -0.3, 0.0, 0.4])  # momentum, vol, beta, sharpe
cartera_topdown = multifactorial.optimizar_topdown(mu, Sigma, rf, X, b_star, W_k)

exp_mom_lowvol = multifactorial.crear_estrategia_momentum_lowvol(nombres_factores)
cart_mom_lowvol = multifactorial.optimizar_topdown(mu, Sigma, rf, X, exp_mom_lowvol, W_k)

exp_quality = multifactorial.crear_estrategia_quality(nombres_factores)
cart_quality = multifactorial.optimizar_topdown(mu, Sigma, rf, X, exp_quality, W_k)

print("Estrategias preparadas")

## Paso 1: Validar Carteras

In [ ]:
estrategias_dict = {
    'Max Sharpe': cartera_max_sharpe,
    'Top-Down': cartera_topdown,
    'Momentum+LowVol': cart_mom_lowvol,
    'Quality': cart_quality
}

nombres_activos = [f'asset{i+1}' for i in range(50)]

print("Validación de Carteras:")
print("=" * 60)

for nombre, cartera in estrategias_dict.items():
    if cartera:
        resultado_validacion = validacion.validar_cartera(cartera['w'], cartera['w_rf'], nombres_activos)
        print(f"\n{nombre}:")
        print(f"  Válida: {resultado_validacion['valida']}")
        if resultado_validacion['errores']:
            for error in resultado_validacion['errores']:
                print(f"  ERROR: {error}")
        if resultado_validacion['advertencias']:
            for adv in resultado_validacion['advertencias']:
                print(f"  ADVERTENCIA: {adv}")

## Paso 2: Calcular Métricas Completas

In [ ]:
# Calcular métricas para todas las estrategias
for nombre, cartera in estrategias_dict.items():
    if cartera:
        metricas = validacion.calcular_metricas_cartera(
            cartera['w'], cartera['w_rf'], mu, Sigma, rf
        )
        cartera.update(metricas)

print("Métricas calculadas para todas las estrategias")

## Paso 3: Comparar Estrategias

In [ ]:
df_comparacion = validacion.comparar_estrategias(estrategias_dict, nombres_activos)

print("Tabla Comparativa de Estrategias:")
print("=" * 80)
print(df_comparacion.to_string(index=False))

## Paso 4: Visualizar Comparación

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sharpe Ratio
axes[0, 0].barh(df_comparacion['estrategia'], df_comparacion['sharpe'])
axes[0, 0].set_xlabel('Sharpe Ratio')
axes[0, 0].set_title('Sharpe Ratio por Estrategia')
axes[0, 0].grid(True, alpha=0.3)

# Concentración (Herfindahl)
axes[0, 1].barh(df_comparacion['estrategia'], df_comparacion['herfindahl'])
axes[0, 1].set_xlabel('Índice de Concentración')
axes[0, 1].set_title('Concentración por Estrategia')
axes[0, 1].grid(True, alpha=0.3)

# Número de activos
axes[1, 0].barh(df_comparacion['estrategia'], df_comparacion['n_activos'])
axes[1, 0].set_xlabel('Número de Activos (>1%)')
axes[1, 0].set_title('Diversificación por Estrategia')
axes[1, 0].grid(True, alpha=0.3)

# Riesgo vs Rentabilidad
axes[1, 1].scatter(df_comparacion['volatilidad']*100, 
                   df_comparacion['rentabilidad']*100, s=200, alpha=0.7)
for i, row in df_comparacion.iterrows():
    axes[1, 1].annotate(row['estrategia'], 
                       (row['volatilidad']*100, row['rentabilidad']*100),
                       fontsize=8, ha='center')
axes[1, 1].set_xlabel('Volatilidad (%)')
axes[1, 1].set_ylabel('Rentabilidad (%)')
axes[1, 1].set_title('Riesgo vs Rentabilidad')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('modulo5_comparacion.png', dpi=300, bbox_inches='tight')
plt.show()

## Paso 5: Seleccionar Mejor Estrategia

In [ ]:
mejor_nombre, mejor_cartera = validacion.seleccionar_mejor_estrategia(estrategias_dict, criterio='sharpe')

print(f"Mejor estrategia según Sharpe Ratio: {mejor_nombre}")
print(f"  Sharpe: {mejor_cartera['sharpe']:.4f}")
print(f"  Rentabilidad: {mejor_cartera['rentabilidad']*100:.2f}%")
print(f"  Volatilidad: {mejor_cartera['volatilidad']*100:.2f}%")
print(f"  Concentración: {mejor_cartera['herfindahl']:.4f}")
print(f"  Número de activos: {mejor_cartera['n_activos']}")

## Paso 6: Exportar Pesos Finales

In [ ]:
df_pesos = validacion.exportar_pesos(mejor_cartera['w'], nombres_activos, 
                           ruta_archivo='pesos_finales.csv')

print("\nTop 10 activos por peso:")
print(df_pesos.head(10))

## Resumen del módulo 5validacion

El módulo 5validacion ha completado exitosamente:

1. Validación de todas las carteras (restricciones cumplidas)
2. Cálculo de métricas completas (Sharpe, concentración, diversificación)
3. Comparación sistemática de estrategias
4. Selección de la mejor estrategia según Sharpe Ratio
5. Exportación de pesos finales para entrega

La cartera seleccionada está lista para ser evaluada en retornos futuros.